In [3]:
#hts_ts_list

import json
import pandas as pd
import numpy as np
from datetime import datetime
from os import listdir
from os.path import isfile, join

mypath = 'data/testing-folder-with-5-tweet-files/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

min = 2
trend_multiple = 3

def normalize(df):
    # 180/2 = 90
    return (df+1) / (df + 1).rolling(90).mean()

#Emphasize large spikes
def emphasize(df):
    #rolling 2
    alpha = 1.2
    em = lambda x: ( np.abs(x[0] - x[1])**alpha )
    df1 = df.rolling(2).apply(em)
    return df1

def smooth(df):
    # 10 min rolling
    r = 5
    return df.rolling(r).sum()

def detect_valid_trend(df, df_raw):
    threshold = 2
    df1 =  ( (df / df.shift().rolling(90).mean() ) > trend_multiple )*1
    no_consec = lambda x: ( 0 if sum(x) > 1 else sum(x))
    df_trend = df1
    #df_trend = df1.rolling(90).apply(no_consec)
    df_valid = ( (df_raw.shift().rolling(90).sum() > 200) & (df_raw.shift().rolling(90).mean() > threshold)  )*1
    df_trend = df_trend * df_valid
    return df_valid, df_trend

def count_hts():
    hts = {}
    for fpath in onlyfiles:
        with open(mypath+fpath) as f:
            for line in f:
                tweet = json.loads(line)
                for ht in tweet['hashtags']:
                    ht_name = ht['text'].lower()
                    if ht_name in hts:
                        hts[ht_name] += 1
                    else:
                        hts[ht_name] = 1
    hts = sorted(hts.items(), key=lambda x: x[1], reverse=True)
    hts_out = {}
    for x in hts:
        if x[1] < 200:
            break
        hts_out[x[0]] = x[1]
    return hts_out

def get_ts_from_hts(hts):
    hts_ts_list = {}
    for fpath in onlyfiles:
        with open(mypath+fpath) as inputF:
            #print(fpath)
            for line_index, line in enumerate(inputF):
                tweet = json.loads(line)
                for ht in tweet['hashtags']:
                    ht_name = ht['text'].lower()
                    if ht_name not in hts:
                        continue
                    if ht_name not in hts_ts_list:
                        hts_ts_list[ht_name] = []
                    tweet_time = datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y')
                    tt = tweet_time.strftime("%Y%m%d-%H%M%S")
                    hts_ts_list[ht_name].append(tt)
    return hts_ts_list
    '''with open('hts_ts_list_new','a') as f:
        for k,v in hts_ts_list.items():
            f.write(json.dumps([k,v])+'\n')'''

def get_hts_3h_samples2(d):
    o_list = []
    ht_name = '#' + d[0]
    timestamp_list = [ datetime.strptime(x, "%Y%m%d-%H%M%S") for x in d[1] ]
    freq = str(min) + 'min'
    # 148 days, from/includinig sep25, to/not including feb 20
    rng = pd.date_range('25/09/2017', periods=148*24*60/min, freq=freq)
    df = pd.DataFrame(index=rng)
    kw = {ht_name:pd.Series(np.zeros(len(rng)),dtype=np.int, index=rng)}
    df = df.assign(**kw)
    
    i = 0
    for ts in timestamp_list:
        while rng[i] < ts:
            i += 1
        i -= 1
        df.at[rng[i],ht_name] += 1
        '''for i in range(0,len(rng)-1):
            if ts >= rng[i] and ts < rng[i+1]:
                df.at[rng[i],ht_name] += 1'''
    
    df0 = df
    df1 = normalize(df0)
    df2 = emphasize(df1)
    df3 = smooth(df2)
    df_valid, df_trend = detect_valid_trend(df3, df0)
    df_nonTrend = df_valid
    tag_name = ht_name
    
    num_output = 0
    for i in range(90, len(rng)-90):
        # trend first
        #if sum(df_valid[ht_name].tolist()[i-90:i-1]) == 0 and df_valid.at[rng[i],ht_name]:
        if df_valid.at[rng[i],ht_name]:
            isTrend = 0 
            if df_trend.at[rng[i],ht_name] == 1 and sum(df_trend[ht_name].tolist()[i-90:i-1]) == 0:
                isTrend = 1
                for j in range(i+1,i+90):
                    df_trend.at[rng[j],ht_name] = 0
                    df_valid.at[rng[j],ht_name] = 0
            elif df_nonTrend.at[rng[i],ht_name] == 1 and sum(df_nonTrend[ht_name].tolist()[i-90:i-1]) == 0:
                isTrend = 0
                for j in range(i+1,i+90):
                    df_nonTrend.at[rng[j],ht_name] = 0
                    df_valid.at[rng[j],ht_name] = 0
            else:
                continue
                
            num_output += 1
                
            #df0 = df[start_dt:start_dt+timedelta(minutes=180)]
            df_raw = df0[rng[i-90]:rng[i]]
            df_nor = df1[rng[i-90]:rng[i]]
            df_em = df2[rng[i-90]:rng[i]]
            df_s = df3[rng[i-90]:rng[i]]
            
            obj = {
                        'ht_name':ht_name,
                        'ts_start':rng[i-90].strftime("%Y%m%d-%H%M%S"),
                        'isTrend':isTrend,
                        'ts_full':[x.strftime("%Y%m%d-%H%M%S") for x in rng[i-90:i] ],
                        'df_raw':df_raw[ht_name].tolist(),
                        'df_nor':df_nor[ht_name].tolist(),
                        'df_em':df_em[ht_name].tolist(),
                        'df_smoothed':df_s[ht_name].tolist() ,
            }
            o_list.append(obj)
            '''with open('raw_to_smoothed_hts.txt','a') as outF:
                outF.write(json.dumps(o_list) + '\n')'''
                
    return o_list

def get_hts_3h_samples(hts_ts_list):
    hts_3h_samples = []
    for k,v in hts_ts_list.items():
        d = [k, v]
        o_list = get_hts_3h_samples2(d)
        for obj in o_list:
            hts_3h_samples.append(obj)
    return hts_3h_samples
        
def get_ts_from_cluster(cluster_raw):
    # cluster_raw = [[" daily", " thanks", " latest"], [" nsfw", " dating", " adultwebcamdate"] ]
    word_to_cluster = {}
    cluster = {}
    
    for c in cluster_raw:
        c0 = c[0].strip()
        c1 = c[1].strip()
        c2 = c[2].strip()
        cluster_id = c0 + c1 + c2
        word_to_cluster[c0] = cluster_id
        word_to_cluster[c1] = cluster_id
        word_to_cluster[c2] = cluster_id
        cluster[cluster_id] = {}
        cluster[cluster_id]['words'] = [c0, c1, c2]
    
    cluster_ts_list = {}
    for fpath in onlyfiles:
        with open(mypath+fpath) as inputF:
            for line_index, line in enumerate(inputF):
                tweet = json.loads(line)
                words_in_text = tweet['text'].lower().split()
                for w in words_in_text:
                    if w not in word_to_cluster:
                        continue
                    cluster_id = word_to_cluster[w]
                    if cluster_id not in cluster_ts_list:
                        cluster_ts_list[cluster_id] = {}
                        cluster_ts_list[cluster_id]['words'] = cluster[cluster_id]['words']
                        cluster_ts_list[cluster_id]['ts'] = []
                    tweet_time = datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y')
                    tt = tweet_time.strftime("%Y%m%d-%H%M%S")
                    cluster_ts_list[cluster_id]['ts'].append(tt)
                    break
       
    return cluster_ts_list
    '''with open('clusterfyp_withpca_optimised_ts_list_new','a') as f:
        for k,v in cluster_ts_list.items():
            f.write(json.dumps([k,v['words'],v['ts']])+'\n')'''
    
def get_cluster_3h_samples2(d):
    o_list = []
    cluster_id = '#' + d[0]
    words = d[1]
    timestamp_list = [ datetime.strptime(x, "%Y%m%d-%H%M%S") for x in d[2] ]
    freq = str(min) + 'min'
    # 148 days, from/includinig sep25, to/not including feb 20
    rng = pd.date_range('25/09/2017', periods=148*24*60/min, freq=freq)
    df = pd.DataFrame(index=rng)
    kw = {cluster_id:pd.Series(np.zeros(len(rng)),dtype=np.int, index=rng)}
    df = df.assign(**kw)
    
    i = 0
    for ts in timestamp_list:
        while rng[i] < ts:
            i += 1
        i -= 1
        df.at[rng[i],cluster_id] += 1
        
    df0 = df
    df1 = normalize(df0)
    df2 = emphasize(df1)
    df3 = smooth(df2)
    df_valid, df_trend = detect_valid_trend(df3, df0)
    df_nonTrend = df_valid
    tag_name = cluster_id
    
    num_output = 0
    for i in range(90, len(rng)-90):
        # trend first
        #if sum(df_valid[ht_name].tolist()[i-90:i-1]) == 0 and df_valid.at[rng[i],ht_name]:
        if df_valid.at[rng[i],cluster_id]:
            isTrend = 0 
            if df_trend.at[rng[i],cluster_id] == 1 and sum(df_trend[cluster_id].tolist()[i-90:i-1]) == 0:
                isTrend = 1
                for j in range(i+1,i+90):
                    df_trend.at[rng[j],cluster_id] = 0
                    df_valid.at[rng[j],cluster_id] = 0
            elif df_nonTrend.at[rng[i],cluster_id] == 1 and sum(df_nonTrend[cluster_id].tolist()[i-90:i-1]) == 0:
                isTrend = 0
                for j in range(i+1,i+90):
                    df_nonTrend.at[rng[j],cluster_id] = 0
                    df_valid.at[rng[j],cluster_id] = 0
            else:
                continue
                
            num_output += 1
                
            #df0 = df[start_dt:start_dt+timedelta(minutes=180)]
            df_raw = df0[rng[i-90]:rng[i]]
            df_nor = df1[rng[i-90]:rng[i]]
            df_em = df2[rng[i-90]:rng[i]]
            df_s = df3[rng[i-90]:rng[i]]
            
            obj = {
                        'cluster_id':cluster_id,
                        'cluster_words':words,
                        'ts_start':rng[i-90].strftime("%Y%m%d-%H%M%S"),
                        'isTrend':isTrend,
                        'ts_full':[x.strftime("%Y%m%d-%H%M%S") for x in rng[i-90:i] ],
                        'df_raw':df_raw[cluster_id].tolist(),
                        'df_nor':df_nor[cluster_id].tolist(),
                        'df_em':df_em[cluster_id].tolist(),
                        'df_smoothed':df_s[cluster_id].tolist() ,
            }
            o_list.append(obj)
            '''with open('clusterfyp_withpca_optimised_raw_plus_preprocessed_multiple_'+str(trend_multiple),'a') as outF:
                outF.write(json.dumps(o_list) + '\n')'''

    return o_list

def get_cluster_3h_samples(cluster_ts_list):
    cluster_3h_samples = []
    for k,v in cluster_ts_list.items():
        d = [k, v['words'], v['ts']]
        o_list = get_cluster_3h_samples2(d)
        for obj in o_list:
            cluster_3h_samples.append(obj)
    return cluster_3h_samples

def main():
    hts_filtered = count_hts()
    hts_ts_list = get_ts_from_hts(hts_filtered)
    hts_3h_samples = get_hts_3h_samples(hts_ts_list)
    for s in hts_3h_samples:
        print('ht_name:{} ts_start:{} isTrend:{}'.format(s['ht_name'],s['ts_start'],s['isTrend']))
    
    # Winson need to make the cluster
    #cluster = None
    #cluster_ts_list = get_ts_from_cluster(cluster)
    #cluster_3h_samples = get_cluster_3h_samples(cluster_ts_list)
                    
                    
if __name__ == '__main__':
    main()
        

ht_name:#venezuela ts_start:20180202-132400 isTrend:0
ht_name:#venezuela ts_start:20180202-162600 isTrend:0
ht_name:#venezuela ts_start:20180202-192800 isTrend:0
ht_name:#venezuela ts_start:20180202-223000 isTrend:0
ht_name:#jimin ts_start:20180202-132200 isTrend:0
ht_name:#jimin ts_start:20180202-162400 isTrend:0
ht_name:#jimin ts_start:20180202-202200 isTrend:0
ht_name:#iheartawards ts_start:20180202-131600 isTrend:1
ht_name:#iheartawards ts_start:20180202-161800 isTrend:0
ht_name:#iheartawards ts_start:20180202-192000 isTrend:0
ht_name:#iheartawards ts_start:20180202-222200 isTrend:0
ht_name:#iheartawards ts_start:20180203-012400 isTrend:0
ht_name:#iheartawards ts_start:20180203-042600 isTrend:0
ht_name:#iheartawards ts_start:20180203-072800 isTrend:0
ht_name:#bestfanarmy ts_start:20180202-132800 isTrend:0
ht_name:#bestfanarmy ts_start:20180202-163000 isTrend:0
ht_name:#btsarmy ts_start:20180202-133000 isTrend:0
ht_name:#btsarmy ts_start:20180202-163200 isTrend:0
ht_name:#liar ts_st